# 引言

本章关注于Python在函数签名中的类型注解(Type Hints)。在Python中引入了显式的类型注解，可以为函数参数、返回值、变量等添加类型提示。主要目的在于帮助开发工具通过静态检查发现代码中的Bug。

# 新内容简介

全新内容。

由于静态类型系统的限制，PEP 484 引入了渐进类型系统(gradual type system)。

# 什么是渐进类型

PEP 484 引入了渐进类型系统。一个渐进类型系统：
* 是可选的 默认情况下，类型检查器不应该为没有类型注解的代码发出告警。当类型检查器不能决定对象的类型时，会假设它为`Any`类型。`Any`类型被认为与所有其他类型兼容。
* 不在运行时捕获类型错误 类型注解由静态类型检查器使用，IDE会发出告警。但它们不会防止不一致的值传递到函数或在运行时赋值给变量。
* 不增强性能 类型注解提供的数据理论上允许在生成的字节码上优化，但这种优化未在任何Python运行时中实现。



渐进类型最有用的特征在于注解总是可选的。

在静态类型系统中，大多数类型约束很容易表达，但还有一些是笨重的、一些是难的，还有一小部分是不可能的。你很可能会编写一段优秀的Python代码，具有良好的测试覆盖率和测试通过率，但仍无法增加使类型检查器满意的类型注解。

类型注解在所有级别上都是可选的：你的整个包可以没有类型注解，当你引入该包到使用类型注解的模块时可以静默类型检查器，也可以增加特殊的评论来类型检查器忽略代码中特定的行。

# 渐进类型实战

由一个简单的函数开始，然后逐渐增加类型注解。
`messages.py`
```py
def show_count(count, word):
    if count == 1:
        return f'1 {word}'
    count_str = str(count) if count else 'no'
    return f'{count_str} {word}s'
```


In [3]:
def show_count(count, word):
    if count == 1:
        return f'1 {word}'
    count_str = str(count) if count else 'no'
    return f'{count_str} {word}s'


show_count(99, 'bird')

'99 birds'

In [4]:
show_count(1, 'bird')

'1 bird'

In [5]:
show_count(0, 'bird')

'no birds'

## 由Mypy开始

在`messages.py`上运行`mypy`命令来开始类型检查：

In [10]:
! pip install mypy

In [9]:
! mypy ./messages/no_hints/messages.py

Success: no issues found in 1 source file


此时Mypy没有发现任何问题。

如果一个函数签名没有任何注解，Mypy默认会忽略它。下面，增加`pytest`单元测试，该代码在`messages_test.py`中：
```py
from pytest import mark

from messages import show_count

@mark.parametrize('qty, expected', [
    (1, '1 part'),
    (2, '2 parts'),
])
def test_show_count(qty, expected):
    got = show_count(qty, 'part')
    assert got == expected

def test_show_count_zero():
    got = show_count(0, 'part')
    assert got == 'no parts'
```

下面开始增加类型注解。

## 让Mypy更严格

命令行可选项`--disallow-untyped-defs`使Mypy标记任何对其所有参数和返回值没有类型注解的函数定义。

使用`--disallow-untyped-defs`在测试文件上产生三个error和一个note：

In [13]:
! mypy  --disallow-untyped-defs ./messages/no_hints/messages_test.py

messages\no_hints\messages.py:1: error: Function is missing a type annotation
messages\no_hints\messages_test.py:9: error: Function is missing a type annotation
messages\no_hints\messages_test.py:13: error: Function is missing a return type annotation
messages\no_hints\messages_test.py:13: note: Use "-> None" if function does not return a value
Found 3 errors in 2 files (checked 1 source file)


对于渐进类型的第一步，我倾向于使用另一个可选项：`--disallow-incomplete-defs`。初始时，它不会抱怨任何问题：

In [14]:
! mypy  --disallow-incomplete-defs ./messages/no_hints/messages_test.py

Success: no issues found in 1 source file


接着，我可以仅为`message.py`中的`show_count`增加返回类型：
```py
def show_count(count, word) -> str:
```
这足以让Mypy查看它了。使用前面同样的命令行去检查`messages_test.py`会让Mypy再次查看`messages.py`：

In [15]:
! mypy  --disallow-incomplete-defs ./messages/no_hints/messages_test.py

messages\no_hints\messages.py:1: error: Function is missing a type annotation for one or more arguments
Found 1 error in 1 file (checked 1 source file)


现在我们可以逐渐为函数增加类型注解，而不会得到我们没有标注的告警。下面是能满足Mypy的完整带注解签名：
```py
def show_count(count: int, word: str) -> str:
```

In [16]:
! mypy  --disallow-incomplete-defs ./messages/no_hints/messages_test.py

Success: no issues found in 1 source file


除了在命令行中使用可选项之外，还可以把你想要的可选项写到Mypy配置文件中。可以有全局设定和模块设定。
下面是一个简单的`mypy.ini`例子：
```
[mypy]
python_version = 3.9
warn_unused_configs = True
disallow_incomplete_defs = True
```

## 默认的参数值

示例中的`show_count`函数只用于(英文)常规名词。如果复数不能通过附加's'来拼写，我们应该让用户提供复数形式，像这样：
```py
>>> show_count(3, 'mouse', 'mice')
'3 mice'
```

让我们做一点"类型驱动开发"。首先，我们添加一个使用第三个参数的测试。不要忘记向测试函数中添加返回类型提示，否则Mypy将不会检查它。
```py
def test_irregular() -> None:
    got = show_count(2, 'child', 'children')
    assert got == '2 children'
```



In [18]:
! mypy  ./messages/hints_2/messages_test.py

messages\hints_2\messages_test.py:21: error: Too many arguments for "show_count"
Found 1 error in 1 file (checked 1 source file)


现在我们编辑`show_count`，增加可选的`plural`参数：
```py
def show_count(count: int, singular: str, plural: str = '') -> str:
    if count == 1:
        return f'1 {singular}'
    count_str = str(count) if count else 'no'
    if not plural:
        plural = singular + 's'
    return f'{count_str} {plural}'
```



In [19]:
! mypy  ./messages/hints_2/messages_test.py

Success: no issues found in 1 source file


现在Mypy就不会报错了。

## 使用None作为默认值

在上面，参数`plural`被注解为`str`，它的默认值为`''`，所以没有类型冲突。

这种方法不错，但在其他情况下，`None`是更好的默认值。如果可选的参数需要一个可变类型，那么`None`是唯一合理的默认值。

为了让`None`作为`plural`参数的默认值，新的签名变成：
```py
from typing import Optional

def show_count(count: int, singular: str, plural: Optional[str] = None) -> str:
```

我们拆开来看：
* `Optional[str]` 意味着`plural`可以为`str`或`None`
* 你必须显示地提供默认值` = None`

如果没有为`plural`赋予默认值，Python运行时会将它看成必传参数。记住：在运行时，忽略类型注解。

注意我们需要从`typing`引入`Optional`。在导入类型时，最好使用`from typing import X`的语法来减少函数签名的长度。

> `Optional`并不是一个很好的名称，因为它不会使参数成为可选的。使参数成为可选的原因是为该参数分配一个默认值。`Optional[str]`仅表示：此参数的类型可以是`str`或`None`T类型。

# 类型被定义来支持操作

实际上，将受支持的操作集合看成一种类型的定义特征更有用。

比如，从可用的操作角度来看，在下面的函数中，`x`的有效类型是什么？
```py
def double(x):
    return x * 2
```

`x`参数类型可能是数值(`int`、`complex`、`Fraction`和`numpy.uint32`等)，但也可能是一个序列(`str`、`tuple`、`list`和`array`)，一个N维的`numpy.array`，或其他实现或继承了`__mul__`方法能接受`int`参数的类型。

然而，考虑下面注解的`double`方法。先忽略返回类型，我们关注于参数类型：
```py
from collections import abc
def double(x: abc.Sequence):
    return x * 2
```

类型检查器会拒绝该代码。如果你告诉Mypy `x`是类型`abc.Sequence`，它会标记`x * 2`为错误因为`abc.Sequence`没有实现或继承`__mul__`方法。在运行时，该代码可以在具体的序列，比如`str`、`tuple`、`list`、`array`等，以及数值上正常运行，因为运行时忽略类型注解。但类型检查器只关心显示声明的，而`abc.Sequence`没有`__mul__`。

这就是本小节的标题是“类型被定义来支持操作”的原因。Python运行时接收任意类型作为`x`参数。`x * 2`可能正常执行，或抛出`TypeError`(如果`x`不支持该操作)。相比之下，Mypy会认为注解版本下`doulbe`中的`x * 2`是错误的，因为`x`的类型`abc.Sequence`不支持该操作。

在渐进类型系统中，我们有两种不同的观点相互作用：
* 鸭子类型(Duck typing) 这种观点被Smalltalk——首创的面向对象语言——以及Pyhton,JS,Ruby采纳。对象必须指定类型，但变量(包括参数)是无类型的。实际上，对象定义的类型不重要，重要的是它实际支持的操作。如果我们能调用`birdie.quack()`，那么`birdie`在该上下文中是鸭子。根据定义，只有在运行时尝试对对象进行操作时才强制鸭子类型。这比名义类型更灵活，但代价是在运行时允许更多的错误。
* 名义类型(Nominal typing) 该观点被C++、Java和C#采纳，也被注解的Python支持。对象和变量都有类型。但对象只在运行时存在，然后类型检查器只关心由类型注解标注的变量(和参数)的源码。如果`Duck`是`Bird`的子类，你可以分配一个`Duck`实例到标记为`Bird`的参数`birdie`。但在函数体中，类型检查器认为调用`birdie.quack()`是非法的，因为`birdie`名义上是`Bird`，而该类没有提供`.quack()`方法。在运行时的实际参数是否为`Duck`并不重要，因为名义类型化是静态强制的。类型检查器不会运行任何程序，只会读取源代码。这比鸭子类型更严格，鸭子类型可以早在构建pipeline时就可以捕获一些bug，甚至在IDE中输入代码时。

下面的示例故意同时对比鸭子类型和名义类型，以及静态类型检查和运行时行为。
`birds.py`:
```py
class Bird:
    pass

class Duck(Bird):  # Duck是Bird子类
    def quack(self):
        print('Quack!')

def alert(birdie):  # alert没有类型注解，所以类型检查忽略它
    birdie.quack()

def alert_duck(birdie: Duck) -> None:  # alert_duck接收一个类型为Duck的参数
    birdie.quack()

def alert_bird(birdie: Bird) -> None:  # alert_bird接收一个类型为Bird的参数
    birdie.quack()
```
用Mypy进行类型检查，我们可以看到一个错误：


In [1]:
! mypy  ./birds/birds.py

birds\birds.py:15: error: "Bird" has no attribute "quack"
Found 1 error in 1 file (checked 1 source file)


我们来分析源码，Mypy发现`alert_bird`是有问题的：类型注解声明`birdie`参数的类型为`Bird`，但函数体调用的是`birdie.quack()`——`Bird`类没有这样的方法。

现在我们试着在`daffy.py`中使用`birds`模块：
`daffy.py`:
```py
from birds import *

daffy = Duck()
alert(daffy)       # 有效调用，alert没有类型注解
alert_duck(daffy)  # 有效调用，alert_duck接收Duck参数，daffy是Duck
alert_bird(daffy)  # 有效调用，alert_bird接收Bird参数，daffy也是Bird——Duck的超类
```

在`daffy.py`上运行Mypy抛出同样的错误：

In [2]:
! mypy  ./birds/daffy.py

birds\birds.py:15: error: "Bird" has no attribute "quack"
Found 1 error in 1 file (checked 1 source file)


但是Mypy认为`daffy.py`本身并没有什么问题：这三个函数调用没问题。
现在，如果你运行`daffy.py`,你会得到：

In [3]:
! python ./birds/daffy.py

Quack!
Quack!
Quack!


所有方法都没问题。
在运行时，Python不会关心声明类型。它只用鸭子类型。Mypy在`alert_bird`上标记了一个错误，但通过`daffy`在运行时调用它却没问题。
刚开始让很多Pythonistas惊讶：一个静态类型检查器有时会在我们知道可以执行的程序中发现错误。

然而，如果几个月后你的任务是扩展这个`bird`的例子，你可能会感激Mypy。考虑一下这个`woody.py`模块，它也使用了`birds`：
`woody.py`:
```py
from birds import *

woody = Bird()
alert(woody)
alert_duck(woody)
alert_bird(woody)
```



In [5]:
! mypy  ./birds/woody.py

birds\birds.py:15: error: "Bird" has no attribute "quack"
birds\woody.py:5: error: Argument 1 to "alert_duck" has incompatible type "Bird"; expected "Duck"
Found 2 errors in 2 files (checked 1 source file)


Mypy发现了两个错误，第一个是在`birds.py`中：`alert_bird`内的`birdie.quack()`，我们之前见过。
第二个是在`woody.py`中：`woody`是`Bird`的实例，所以调用`alert_duck(woody)`是无效的，因为该函数需要的是`Duck`。每个`Duck`都是`Bird`，但并不是每个`Bird`都是`Duck`。

在运行时，`woody.py`中没有一个成功执行的。

In [11]:
from birds.birds import *
woody = Bird()
alert(woody) # Mypy无法检测到此错误，因为alert中没有类型提示。

AttributeError: 'Bird' object has no attribute 'quack'

In [13]:
alert_duck(woody) # Mypy报告了该问题：第一个参数的类型不兼容，期望Duck

AttributeError: 'Bird' object has no attribute 'quack'

In [14]:
alert_bird(woody) # Mypy告诉我们alert_bird的函数体是错误的：Bird没有属性quack

AttributeError: 'Bird' object has no attribute 'quack'

这个小实验表明，鸭子类型更容易理解，也更灵活，但允许不受支持的操作在运行时导致错误。标称输入在运行前检测错误，但有时会拒绝实际运行的代码——例如上面示例中的调用`allalert_bird(daffy)`。即使它有时可以工作，`alert_bird`函数的命名也是错误的：它的主体确实需要一个支持`.quack()`方法的对象，而`Bird`没有这个方法。

在这个例子中，函数是单行的。但在实际代码中，它们可以更长；他们可以将`birdie`参数传递给更多的函数，而`birdie`参数的起源可能是许多函数调用，这使得很难查明运行时错误的原因。类型检查器可以防止在运行时发生许多这样的错误。

# 注解可用的类型

有很多Python类型可用在类型注解上，但这里有一些约束和建议。除此之外，`typing`模块引入了令人惊讶的特殊的具有语义的结构。

## Any类型

任何渐进类型系统的基石是`Any`类型，也被称为动态类型。当类型检查器看到未注解的函数：
```py
def double(x):
    return x * 2
```
它会假设为：
```py
def double(x: Any) -> Any:
    return x * 2
```
这意味着`x`和返回值可以是任何类型，包括不同的类型。`Any`被认为可以支持任何操作。

`Any`与`object`对比，考虑下面的签名：
```py
def double(x: object) -> object:
```
该函数也支持任何类型的参数，因为所有类型都是`object`子类。

然而，类型检查器会拒绝该函数。因为`object`不支持`__mul__`操作：

In [15]:
! mypy  ./double/double_object.py

double\double_object.py:2: error: Unsupported operand types for * ("object" and "int")
Found 1 error in 1 file (checked 1 source file)


更通用的类型具有更窄(narrower)的接口，比如，它们支持更少的操作。`object`比`abc.Sequence`实现的操作更少，而后者比`abc.MutableSequence`实现的更少。

但`Any`是一个魔法类型，它位于类型层次结构的顶部和底部。它同时也是最通用的类型——所以一个参数`n: Any`支持任何类型的值——同时最具体的类型，支持任何可能的操作。至少，这是类型检查器理解的`Any`。

当然，没有类型可以支持所有可能的操作，所以使用`Any`可以阻止类型检查器完成其核心任务：在程序崩溃和运行时异常之前检测潜在的非法操作。

### 子类型vs一致
传统的面向对象名义类型系统依赖子类型关系。给定一个类`T1`和一个子类`T2`，那么`T2`是`T1`的子类型。
考虑下面的代码：

```py
class T1:
    pass

class T2(T1):
     pass
    
def f1(p: T1) -> None:
    pass

o2 = T2()

f1(o2) # OK
```

`f1(o2)`是里氏替代原则的应用。Barbara Liskov实际上定义的是关于支持的操作的子类型：如果`T2`类型的对象替换了`T1`类型的对象，并且该程序仍然正确地行为，则`T2`是`T1`的子类型。

继续前面的代码，下面显示违背了里氏替代原则：
```py
def f2(p: T2) -> None:
    pass

o1 = T1()

f2(o1) # 类型错误
```

从支持操作的角度来看，这是完全有意义的：作为子类，`T2`继承且必须支持`T1`的所有操作。所以一个`T2`的实例能用在任何期望为`T1`实例的地方。但反过来不一定成立：`T2`可能实现了额外的方法，所以`T1`的实例就可能无法应用到`T2`能用的地方。

在渐进类型系统中，有另外一种关系：一致(consistent-with)，适用于子类型的适用范围，并对`Any`类型有特殊规定。

一致的规则是：
1. 给定类型`T1`和子类型`T2`，那么`T2`与`T1`是一致的。
2. 所有的类型与`Any`都是一致的：你可以传递任何类型的对象到声明为`Any`类型的参数。
3. `Any`与所有类型一致：你总可以传递`Any`类型到期望其他类型的参数。

下面是解释规则2,3的代码：
```py
def f3(p: Any) -> None:
    ...
    
o0 = object()
o1 = T1()
o2 = T2()
f3(o0) #
f3(o1) # all OK: rule #2
f3(o2) #

def f4(): # 隐式返回类型 : `Any`
    ...
o4 = f4() # 引用的类型: `Any`
f1(o4) #
f2(o4) # all OK: rule #3
f3(o4) #
```

所有的渐进类型系统都需要一个像`Any`这样的通配符类型。


## 简单类型和类

简单类型像`int`,`float`,`str`和`bytes`可以直接用于类型注解。标准库中的具体类、外部包或用户定义的类也能用于类型注解。

抽象类可用在类型注解。

在类中，一致(consistent-with)像子类型那样定义：一个子类与所有它的父类一致。

然而，实用优于纯粹，所以有一个重要的例外，如下所述。

** `int`与`complex`一致

内置类型`int`、`float`、`complex`类型之间不存在名义的子类型关系：它们是`object`的直接子类。但是PEP 484声明`int`与`float`是一致的，`float`与`complex`是一致的。它在实践中是有意义的：`int`实现了所有`float`执行的操作，且`int`实现其他操作——位操作，如`&`、`|`、`<<`等。最终的结果是： `int`与`complex`是一致的。对于`i = 3`，`i.rea`l是`3`，`i.imag`是`0`。

## Optional和Union类型

`Optional`解决了`None`作为默认值的问题：
```py
from typing import Optional
def show_count(count: int, singular: str, plural: Optional[str] = None) -> str:
```

`Optional[str]`实际上是`Union[str, None]`的简写，后者一位置`plural`的类型可以是`str`或`None`。

**在Python3.10中比`Optional`和`Union`更好的语法**

在Python3.10中，我们可以写`str | bytes`而不是`Union[str, bytes]`。它更简介，且不需要从`typing`中引入`Optional`或`Union`。
对比旧语法和新语法：
```py
plural: Optional[str] = None # 旧
plural: str | None = None # 新
```
`|`操作符也能与`isintance`和`issubclass`一起用来构建第二个参数：`isinstance(x, int | str)`。


内置函数`ord`的签名是`Union`的简单例子——它接收`str`或`bytes`，然后返回`int`:
```py
def ord(c: Union[str, bytes]) -> int: ...
```
还可以以`str`为参数，接收`str`或`float`:
```py
from typing import Union
def parse_token(token: str) -> Union[str, float]:
    try:
        return float(token)
    except ValueError:
        return token
```
可以的话，不要创建返回`Union`类型的函数，因为它们给用户带来额外的负担——强制用户在运行时检查返回值类型。但上面的`parse_token`是一个合理的用例。

`Union[]`至少需要两个类型。嵌套的`Union`类型与扁平的一样：
```py
Union[A, B, Union[C, D, E]]
```
与
```py
Union[A, B, C, D, E]
```
效果相同。

`Union`对于它们之间不一致的类型更有用。比如：`Union[int, float]`是多余的，因为`int`与`float`是一致的。如果你只使用`float`来注释参数，那么它也会接受`int`值。

## 泛型

大多数Python集合都是异构的。例如，你可以在一个列表中混合不同类型的。但是，在实际中，这并不是很有用：如果你将对象放在集合中，你可能希望稍后对它们进行操作，通常这意味着它们必须至少共享一个常见的方法。

泛型可以用类型参数来声明，以指定它们可以处理的元素的类型。

比如，`list`能约束其中元素的类型，比如：

In [19]:
def tokenize(text: str) -> list[str]:
    return text.upper().split()

在Python≥3.9中，这意味着`tokenize`返回一个列表，其中的元素是`str`类型。
注解`stuff: list`和`stuff: list[Any]`是一样的：`stuff`是任意类型的列表。

PEP 585列出了标准库中的接收泛型注解的集合。下面的列表仅显示使用最简单的泛型类型注解，`container[item]`目录：

* list 
* collections.deque 
* abc.Sequence 
* abc.MutableSequence
* set 
* abc.Container
* abc.Set 
* abc.MutableSet
* frozenset 
* abc.Collection

元组和映射支持更复杂的类型注解，我们后面再探讨。

从Python3.10开始，没有好的方式去标注`array.array`，考虑`typecode`构造函数参数，它决定数组中存储的是整数还是浮点数。一个更难的问题是当为数组添加元素时如何检查整数的范围以防止溢出。比如，`typecode='B'`的数组只能存储从0到255的`int`值。因此，Python的静态类型系统并不能满足这个挑战。

## 元组类型

有三种方法来注解元组类型：
* 元组作为记录(record)
* 元组作为带有命名字段的记录
* 元组作为不可变序列

### 元组作为记录

如果你把元组当成记录使用，使用元组内置并通过`[]`定义字段类型。

比如，类型注解为`tuple[str, float, str]`来接收具有城市名、人口和国家的元组：` ('Shanghai', 24.28, 'China')`。

考虑一个函数，它接受一对地理坐标并返回一个地理散列，使用如下：
```py
>>> shanghai = 31.2304, 121.4737
>>> geohash(shanghai)
'wtw3sjq6q'
```

下面显示`geohash`是如何定义的：


In [5]:
from geolib import geohash as gh # type: ignore  # 该注释阻止Mypy报告geolib包没有类型注解

PRECISION = 9

def geohash(lat_lon: tuple[float, float]) -> str:  # lat_lon参数注解为有两个float字段的元组
    return gh.encode(*lat_lon, PRECISION)

### 元组作为带有命名字段的记录

为了注解一个具有很多字段，或指定类型的元组，强烈建议使用`typing.NamedTuple`。


In [6]:
from typing import NamedTuple
from geolib import geohash as gh # type: ignore
PRECISION = 9

class Coordinate(NamedTuple):
    lat: float
    lon: float
    
def geohash(lat_lon: Coordinate) -> str:
    return gh.encode(*lat_lon, PRECISION)

`typing.NamedTuple`是用于元组子类的工厂，所以`Coordinate`与`tuple[float, float]`一致，但反过来不成立。毕竟，`Coordinate`有`NamedTuple`添加的额外方法，像`._asdict()`。

实际上，这意味着传递一个`Coordiante`实例到下面定义的`display`函数是类型安全的：

In [7]:
def display(lat_lon: tuple[float, float]) -> str:
    lat, lon = lat_lon
    ns = 'N' if lat >= 0 else 'S'
    ew = 'E' if lon >= 0 else 'W'
    return f'{abs(lat):0.1f}°{ns}, {abs(lon):0.1f}°{ew}'

### 元组作为不可变序列

为了注解未指定长度作为不可变列表的元组，你必须指定单个类型，然后是`,`和`...`。

比如，`tuple[int, ...]`是一个包含`int`元素的元组。

`...`(省略)表示任何数量的(>=1)元素是可接受的。但没有办法为任意长度的元组指定不同类型的字段。

注解`stuff: tuple[Any, ...]`和`stuff: tuple`是一样的：`stuff`是一个未定长度包含`Any`类型的元组。

下面是一个`columnize`函数，将一个序列转换成未定长度的元组列表，作为表格中的rows和cells：

In [8]:
from collections.abc import Sequence

def columnize(
    sequence: Sequence[str], num_columns: int = 0
) -> list[tuple[str, ...]]:
    if num_columns == 0:
        num_columns = round(len(sequence) ** 0.5)
    num_rows, reminder = divmod(len(sequence), num_columns)
    num_rows += bool(reminder)
    return [tuple(sequence[i::num_rows]) for i in range(num_rows)]

这非常适合于展示列中的项，比如：

In [9]:
animals = 'drake fawn heron ibex koala lynx tahr xerus yak zapus'.split()
table = columnize(animals)
table

[('drake', 'koala', 'yak'),
 ('fawn', 'lynx', 'zapus'),
 ('heron', 'tahr'),
 ('ibex', 'xerus')]

## 泛型映射

泛型映射类型被标注为`MappingType[KeyType, ValueType]`。在Python≥3.9中，内置的`dict`和`collections`以及`collections.abc`中的映射类型接受该注解。对于早期版本，你必须使用`typing.Dict`和其他`typing`模块中的映射类型。

下面显示了一个返回反向索引的函数来按名称搜索Unicode字符的实际用法。

In [31]:
import sys
import re
import unicodedata
from collections.abc import Iterator

RE_WORD = re.compile(r'\w+')
STOP_CODE = sys.maxunicode + 1

def tokenize(text: str) -> Iterator[str]:  # 这是一个生成器函数
    """return iterable of uppercased words"""
    for match in RE_WORD.finditer(text):
        yield match.group().upper()


def name_index(start: int = 32, end: int = STOP_CODE) -> dict[str, set[str]]:
    # 局部变量index被标注了，否则，Mypy会报告： Need type annotation for 'index' (hint: "index: dict[<type>, <type>] = ...")
    index: dict[str, set[str]] = {}  
    for char in (chr(i) for i in range(start, end)):
        # 使用海象运算符保存if表达式中的unicodedata.name(char, '')的结果到name，并根据这个name来进行判断，如果为''，则条件为False
        # unicodedata.name(char, '')返回char对应的名称，比如unicodedata.name('$')='DOLLAR SIGN'
        if name := unicodedata.name(char, ''): 
            # 为名称进行分词
            for word in tokenize(name):
                index.setdefault(word, set()).add(char)
    return index

给定开始和结束Unicode字符码，`name_index`返回一个`dict[str, set[str]]`，这是一个反向索引映射每个单词到一组名称中有该单词的字符。
比如在索引ASCII字符从32到64后，下面是映射到单词'SIGN'和'DIGIT'的字符集，以及如何找到名为'DIGIT EIGHT'的字符。


In [32]:
index = name_index(32, 65)
index['SIGN']

{'#', '$', '%', '+', '<', '=', '>'}

In [33]:
index['DIGIT']

{'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}

In [34]:
index['DIGIT'] & index['EIGHT']

{'8'}

## 抽象基类

